![First Power Bi](.\images\dbachecks-logo.png )
# Checking Agent

Agent Jobs are an important part of a DBAs responsiblity

In [69]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
$null = Reset-DbcConfig

Lets take a look at the checks that have agent in them

In [6]:
Get-DbcCheck Agent


Group    Type        UniqueTag                   AllTags
-----    ----        ---------                   -------                                           
Agent    Sqlinstance DatabaseMailEnabled         DatabaseMailEnabled, CIS, security, Agent         
Agent    Sqlinstance AgentServiceAccount         AgentServiceAccount, ServiceAccount, Agent        
Agent    Sqlinstance DbaOperator                 DbaOperator, Operator, Agent                      
Agent    Sqlinstance FailsafeOperator            FailsafeOperator, Operator, Agent                 
Agent    Sqlinstance DatabaseMailProfile         DatabaseMailProfile, Agent                        
Agent    Sqlinstance FailedJob                   FailedJob, Agent                                  
Agent    Sqlinstance ValidJobOwner               ValidJobOwner, Agent                              
Agent    Sqlinstance AgentAlert                  AgentAlert, Agent                                 
Agent    Sqlinstance JobHistory           

and the configurations

In [11]:
Get-DbcConfig *agent*


Name                                                  Value Description
----                                                  ----- -----------
agent.alert.Job                                       False Agent alert job notification. Ex job t…
agent.alert.messageid                       {823, 824, 825} Agent alert messageid to validate; htt…
agent.alert.Notification                               True Agent alert notification
agent.alert.Severity                      {16, 17, 18, 19…} Agent alert severity to validate; http…
agent.databasemailprofile                                   Name of the Database Mail Profile in S…
agent.dbaoperatoremail                                      Email address of the DBA Operator in S…
agent.dbaoperatorname                                       Name of the DBA Operator in SQL Agent
agent.failedjob.excludecancelled                      False Exclude agent jobs with a status of ca…
agent.failedjob.since                                    30 The maximum 

# Checking Failed Jobs

We have already checked for failed jobs in an earlier notebook but lets do it again

In [18]:
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check FailedJob

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags FailedJob

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing Failed Jobs

    Context Checking for failed enabled jobs since 02/19/2020 14:43:57 on localhost,15592
      [+] CommandLog Cleanup's last run outcome on 69017f5d2a89 is Succeeded 60ms
      [-] DatabaseBackup - SYSTEM_DATABASES - FULL's last run outcome on 69017f5d2a89 is Failed 23ms
        Expected 'Succeeded', because All Agent Jobs should have succeed - you need to investigate the failed jobs, but got Failed.
        169:                                         $psitem.LastRunOutcome | Should -Be "Succeeded" -Because 'All Agent Jobs should have succeed - you need to investigate t

If we check the details for this check we can see in the description

In [21]:
Get-DbcCheck failedjob | fl


Group       : Agent
Type        : Sqlinstance
UniqueTag   : FailedJob
AllTags     : FailedJob, Agent
Config      : app.sqlinstance agent.failedjob.since agent.failedjob.excludecancelled 
Description : Tests that enabled Agent Jobs last outcome was succeeded since specified start date 
              (default 30 days ago) and can exclude cancelled jobs (default false)




That is will check the status of all enabled jobs in the last 30 days and it excludes any jobs that have been cancelled

## Changing configuration
If we want to change that and maybe only check the last 24 hours we can change the config

In [70]:
Set-DbcConfig -Name agent.failedjob.since -Value 1
Set-DbcConfig -Name agent.failedjob.excludecancelled -Value $true
# set the instances to save retyping
Set-DbcConfig -Name app.sqlinstance -Value $SqlInstances
Invoke-DbcCheck -SqlCredential $SqlCredential -Check FailedJob -Show Summary


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags FailedJob
Tests completed in 2.09s
Tests Passed: 4, Failed: 7, Skipped: 0, Pending: 0, Inconclusive: 0 
Name                                                          Value Description
----                                                          ----- -----------
agent.failedjob.since                                             1 The maximum number of days to …
agent.failedjob.excludecancelled                               True Exclude agent jobs with a stat…
app.sqlinstance                  {localhost,15592, localhost,15593} List of SQL Server instances t…



# Operators

We can check that we have operators for our Agent Jobs

In [71]:
Invoke-DbcCheck -SqlCredential $SqlCredential -Check DbaOperator

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags DbaOperator

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing DBA Operators

    Context Testing DBA Operators exists on localhost,15592

  Describing DBA Operators

    Context Testing DBA Operators exists on localhost,15593
Tests completed in 4.11s
Tests Passed: 0, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


That looks weird, No tests run.

## Changing Configuration

Thats because we havent set a config, if we look at the description of the check and the config we can see that we need to set the values

In [73]:
Get-DbcCheck dbaop | fl



Group       : Agent
Type        : Sqlinstance
UniqueTag   : DbaOperator
AllTags     : DbaOperator, Operator, Agent
Config      : app.sqlinstance agent.dbaoperatorname agent.dbaoperatoremail 
Description : Tests that the specified (default blank) DBA Operators exist and have the correct 
              email address




Lets set the values (incorrectly) and run the check again

In [75]:
Set-DbcConfig agent.dbaoperatorname -Value 'The wrong thing'
Set-DbcConfig agent.dbaoperatoremail -Value 'TheWrongEmailAddress@Nope.Com'

Invoke-DbcCheck -SqlCredential $SqlCredential -Check DbaOperator


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags DbaOperator

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing DBA Operators

    Context Testing DBA Operators exists on localhost,15592
      [-] operator name The wrong thing exists 15ms
        Expected collection $null to contain 'The wrong thing', because This Operator is expected to exist, but it was not found.
        92:                                     $psitem | Should -BeIn $Results.Name -Because 'This Operator is expected to exist'
        at <ScriptBlock>, C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1: line 92
      [-] operator email TheWrongEmailAddress@Nope.Com is correct 6ms
        Exp

Lets set the values correctly and run it again

In [76]:
Set-DbcConfig agent.dbaoperatorname -Value 'SQLAdmins'
Set-DbcConfig agent.dbaoperatoremail -Value 'SQLAdmins@TheBeard.local'

Invoke-DbcCheck -SqlCredential $SqlCredential -Check DbaOperator


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags DbaOperator

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing DBA Operators

    Context Testing DBA Operators exists on localhost,15592
      [+] operator name SQLAdmins exists 3ms
      [+] operator email SQLAdmins@TheBeard.local is correct 2ms

  Describing DBA Operators

    Context Testing DBA Operators exists on localhost,15593
      [+] operator name SQLAdmins exists 1ms
      [+] operator email SQLAdmins@TheBeard.local is correct 2ms
Tests completed in 4.7s
Tests Passed: 4, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 
Name                   Value                    Description
----                   -----                    ---

# Database Mail

We can check the Database Mail settings. Lets check the check and see which configurations that uses

In [81]:
Get-DbcCheck DatabaseMailProfile | fl


Group       : Agent
Type        : Sqlinstance
UniqueTag   : DatabaseMailProfile
AllTags     : DatabaseMailProfile, Agent
Config      : app.sqlinstance  agent.databasemailprofile 
Description : Tests that the specified (default blank) Database Mail Profile exists




and check the values of those settings currently

In [84]:
Get-DbcConfigValue agent.databasemailprofile

## Changing Configuration

No response means no value so we had better add a configuration

In [85]:
Set-DbcConfig agent.databasemailprofile -Value SQLDBA


Name                      Value  Description
----                      -----  -----------
agent.databasemailprofile SQLDBA Name of the Database Mail Profile in SQL Agent



In [88]:
Invoke-DbcCheck -SqlCredential $SqlCredential -Check DatabaseMailProfile

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags DatabaseMailProfile

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing Database Mail Profile

    Context Testing database mail profile is set on localhost,15592
      [+] database mail profile on localhost,15592 is SQLDBA 146ms

  Describing Database Mail Profile

    Context Testing database mail profile is set on localhost,15593
      [+] database mail profile on localhost,15593 is SQLDBA 65ms
Tests completed in 987ms
Tests Passed: 2, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


In [89]:
Get-DbaDbMailProfile -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential | Select *


ComputerName                 : localhost
InstanceName                 : MSSQLSERVER
SqlInstance                  : 69017f5d2a89
Parent                       : [localhost,15592]
Description                  : Profile used for database mail for SQL DBA Alerts
ID                           : 1
ForceDeleteForActiveProfiles : True
IsBusyProfile                : False
Name                         : SQLDBA
ParentCollection             : {SQLDBA}
Urn                          : Server[@Name='69017f5d2a89']/Mail/MailProfile[@Name='SQLDBA']
Properties                   : {Name=Description/Type=System.String/Writable=True/Value=Profile 
                               used for database mail for SQL DBA Alerts, 
                               Name=ID/Type=System.Int32/Writable=False/Value=1}
ServerVersion                : 15.0.2000
DatabaseEngineType           : Standalone
DatabaseEngineEdition        : Enterprise
ExecutionManager             : Microsoft.SqlServer.Management.Smo.ExecutionManager
Use

# Job Owners

Maybe you need to make sure that your Agent Jobs are owned by the correct account.

The process is the same

- Set the configuration and then runt he check - remember once you have set a configuration, you can save it (see the other notebooks)

In [91]:
Set-DbcConfig -Name agent.validjobowner.name -Value OldSa 
Invoke-DbcCheck -SqlCredential $SqlCredential -Check ValidJobOwner


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags ValidJobOwner

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing Valid Job Owner

    Context Testing job owners on localhost,15592
      [+] Job CommandLog Cleanup  - owner OldSa should be in this list ( OldSa ) on 69017f5d2a89 4ms
      [+] Job DatabaseBackup - SYSTEM_DATABASES - FULL  - owner OldSa should be in this list ( OldSa ) on 69017f5d2a89 3ms
      [+] Job DatabaseBackup - USER_DATABASES - DIFF  - owner OldSa should be in this list ( OldSa ) on 69017f5d2a89 3ms
      [+] Job DatabaseBackup - USER_DATABASES - FULL  - owner OldSa should be in this list ( OldSa ) on 69017f5d2a89 4ms
      [+] Job DatabaseBackup - USER_DATABASES - LOG  

# Job History

To ensure that you always have the expected amount of job history records to be able to troubleshoot, you might wish to ensure that the agent job retention for number of rows and number of rows per job are validated. No suprises here - set a configuration and run the check.

In [94]:
Set-DbcConfig -Name agent.history.maximumhistoryrows -Value 1000
Set-DbcConfig -Name agent.history.maximumjobhistoryrows -Value 100

Invoke-DbcCheck -SqlCredential $SqlCredential -Check JobHistory


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1', 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\MaintenanceSolution.Tests.ps1' with Tags JobHistory

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing Job History Configuration

    Context Testing job history configuration on localhost,15592
      [+] The maximum job history number of rows configuration should be greater or equal to 1000 on localhost,15592 59ms
      [+] The maximum job history rows per job configuration should be greater or equal to 100 on localhost,15592 3ms

  Describing Job History Configuration

    Context Testing job history configuration on localhost,15593
      [+] The maximum job history number of rows c

# Long Running Jobs



In [97]:
Invoke-DbcCheck -SqlCredential $SqlCredential -Check LongRunningJob

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags LongRunningJob

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing Long Running Agent Jobs

    Context Testing long running jobs on localhost,15592
      [+] There are no running jobs currently on localhost,15592 36ms

  Describing Long Running Agent Jobs

    Context Testing long running jobs on localhost,15593
      [+] There are no running jobs currently on localhost,15593 3ms
Tests completed in 1.23s
Tests Passed: 2, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


# Was the last Agent Job Run Time Too long?

If the last job time was unexpectedly long you might want to check for this. Of course, you know by now that you can check for the configurations as you need them and alter them according to your needs

First lets set up a job to show a failure

In [99]:
$JobName = 'This Job Should Not Run Long' 
$newDbaAgentJobSplat = @{
    SqlInstance = $SqlInstances[0]
    SqlCredential = $SqlCredential
    Job = $JobName
    Description = "A job to show how long a job should run for"
}
New-DbaAgentJob @newDbaAgentJobSplat 

$StepName = 'Run TSQL'
$newDbaAgentJobStepSplat = @{
    SqlInstance = $SqlInstances[0]
    SqlCredential = $SqlCredential
    Job = $JobName
    StepName = $StepName
    Subsystem = 'TransactSql'
    Command = "WAITFOR DELAY '00:00:02';"
}
New-DbaAgentJobStep @newDbaAgentJobStepSplat


Parent                 : [localhost,15592]
Category               : [Uncategorized (Local)]
CategoryType           : 1
CurrentRunRetryAttempt : 0
CurrentRunStatus       : Idle
CurrentRunStep         : 0 (unknown)
DateCreated            : 20/03/2020 17:07:31
DateLastModified       : 20/03/2020 17:07:31
DeleteLevel            : Never
Description            : A job to show how long a job should run for
EmailLevel             : Never
EventLogLevel          : Never
HasSchedule            : False
HasServer              : True
HasStep                : False
IsEnabled              : True
JobID                  : 91d25a5c-a3a2-49eb-a3f9-694e2efe97f3
JobType                : Local
LastRunDate            : 01/01/0001 00:00:00
LastRunOutcome         : Unknown
NetSendLevel           : Never
NextRunDate            : 01/01/0001 00:00:00
NextRunScheduleID      : 0
OperatorToEmail        : 
OperatorToNetSend      : 
OperatorToPage         : 
OriginatingServer      : 69017F5D2A89
OwnerLoginName        

Now lets get a "normal" run of this job a few times

In [100]:
$i = 5
while ($i -gt 0) {
    Start-DbaAgentJob -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential -Job $JobName
    Start-Sleep -Seconds 3
    $i --
}


ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : 69017f5d2a89
Name                   : This Job Should Not Run Long
Category               : [Uncategorized (Local)]
OwnerLoginName         : sqladmin
CurrentRunStatus       : Executing
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 01/01/0001 00:00:00
LastRunOutcome         : Unknown
HasSchedule            : False
OperatorToEmail        : 
CreateDate             : 20/03/2020 17:07:31

ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : 69017f5d2a89
Name                   : This Job Should Not Run Long
Category               : [Uncategorized (Local)]
OwnerLoginName         : sqladmin
CurrentRunStatus       : Executing
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 20/03/2020 17:08:11
LastRunOutcome         : Succeeded
HasSchedule            : False
OperatorToEmail        : 
Cre

Now we can set the job to run for longer

In [101]:
$setDbaAgentJobStepSplat = @{
    SqlInstance = $SqlInstances[0]
    SqlCredential = $SqlCredential
    Job = $JobName
    StepName = $StepName
    Command = "WAITFOR DELAY '00:00:12';"
}
Set-DbaAgentJobStep @setDbaAgentJobStepSplat

and run the job again

In [102]:
Start-DbaAgentJob -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential -Job $JobName


ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : 69017f5d2a89
Name                   : This Job Should Not Run Long
Category               : [Uncategorized (Local)]
OwnerLoginName         : sqladmin
CurrentRunStatus       : Executing
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 20/03/2020 17:08:34
LastRunOutcome         : Succeeded
HasSchedule            : False
OperatorToEmail        : 
CreateDate             : 20/03/2020 17:07:31




Now when we run the check for last job run time we will see a failure for this job

In [103]:
Invoke-DbcCheck -SqlCredential $SqlCredential -Check LastJobRunTime

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1' with Tags LastJobRunTime

Executing script C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1

  Describing Last Agent Job Run

    Context Testing last job run time on localhost,15592
      [+] Job IndexOptimize - USER_DATABASES last run duration should be not be greater than 50 % extra of the average run time on localhost,15592 39ms
      [+] Job DatabaseBackup - SYSTEM_DATABASES - FULL last run duration should be not be greater than 50 % extra of the average run time on localhost,15592 3ms
      [-] Job This Job Should Not Run Long last run duration should be not be greater than 50 % extra of the average run time on localhost,15592 24ms
        Expected the actual value to be less t